<a href="https://colab.research.google.com/github/MahimaMaurya/Indigo/blob/main/Notebooks/4_T5_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install transformers
!pip install datasets
!pip install nltk
!pip install rouge_score
!pip install torch
!pip install sckit-learn

ERROR: Could not find a version that satisfies the requirement sckit-learn (from versions: none)
ERROR: No matching distribution found for sckit-learn


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from transformers import TextDataset, DataCollatorForLanguageModeling
import logging
import warnings
from transformers import T5Tokenizer, AutoModelForQuestionAnswering
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score


In [25]:
df = pd.read_csv('/content/tokenized_output.csv', on_bad_lines='skip')
df1 = pd.read_csv('/content/test_data.csv', on_bad_lines='skip')
df.head()

,question,tokenized_question,stemmed_question,answer,tokenized_answer,stemmed_answer,question_length,answer_length
0,How can a piece of paper folded 103 times be l...,"['how', 'can', 'a', 'piece', 'of', 'paper', 'f...","['how', 'can', 'a', 'piec', 'of', 'paper', 'fo...",It isn’t. It just has a stack height longer t...,"['it', 'isn’t', 'it', 'just', 'has', 'a', 'sta...","['it', 'isn’t', 'it', 'just', 'ha', 'a', 'stac...",81,119
1,Did a landlord ever offer to lower your rent s...,"['did', 'a', 'landlord', 'ever', 'offer', 'to'...","['did', 'a', 'landlord', 'ever', 'offer', 'to'...",Did a landlord ever offer to lower your rent s...,"['did', 'a', 'landlord', 'ever', 'offer', 'to'...","['did', 'a', 'landlord', 'ever', 'offer', 'to'...",72,258
2,"If the Von Karman line is 62 miles above, then...","['if', 'the', 'von', 'karman', 'line', 'is', '...","['if', 'the', 'von', 'karman', 'line', 'is', '...",No. The Karman line has nothing to do with the...,"['no', 'the', 'karman', 'line', 'has', 'nothin...","['no', 'the', 'karman', 'line', 'ha', 'noth', ...",114,759
3,Do soldiers pick up fallen enemy weapons to us...,"['do', 'soldiers', 'pick', 'up', 'fallen', 'en...","['do', 'soldier', 'pick', 'up', 'fallen', 'ene...","It’s against the rules of war so no, if they r...","['it’s', 'against', 'the', 'rules', 'of', 'war...","['it’', 'against', 'the', 'rule', 'of', 'war',...",89,379
4,How do I get my son to stop making music?,"['how', 'do', 'i', 'get', 'my', 'son', 'to', '...","['how', 'do', 'i', 'get', 'my', 'son', 'to', '...","Wha- if he loves making music, let him make mu...","['wha', 'if', 'he', 'loves', 'making', 'music'...","['wha', 'if', 'he', 'love', 'make', 'music', '...",41,244


In [26]:
from datasets import Dataset

In [19]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

df = pd.read_csv('tokenized_output.csv', on_bad_lines='skip')



# Create a small subset
from datasets import load_dataset
dataset = load_dataset('quora')
small_dataset = dataset['train'].train_test_split(test_size=0.1, seed=42)['test']  # Use 10% of the training data

def create_t5_model_and_tokenizer(model_name="t5-small"):
    tokenizer = T5Tokenizer.from_pretrained(model_name)
    model = T5ForConditionalGeneration.from_pretrained(model_name)
    return model, tokenizer

def prepare_training_data(dataset, tokenizer):
    encodings = tokenizer(dataset['tokenized_answer'].tolist(), truncation=True, padding=True)
    return Dataset.from_dict(encodings)

def train_t5_model(model, tokenizer, dataset, output_dir, num_train_epochs=3, per_device_train_batch_size=2, warmup_steps=10, weight_decay=0.01, logging_dir=None):
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        warmup_steps=warmup_steps,
        weight_decay=weight_decay,
        logging_dir=logging_dir,
        logging_steps=100,  # Log less frequently
        save_steps=500,  # Save less frequently
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset
    )

    trainer.train()
    return trainer.model

# Ensure GPU is enabled in Colab
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Example usage
model, tokenizer = create_t5_model_and_tokenizer()
model.to(device)
train_data = prepare_training_data(df, tokenizer)

output_dir = "/content/fine_tuned_T5_model"
logging_dir = "/content/logging_dir"

trained_model = train_t5_model(model, tokenizer, train_data, output_dir, logging_dir=logging_dir)

Using device: cpu


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Step,Training Loss


KeyboardInterrupt: 

In [ ]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForLanguageModeling

# def create_t5_model_and_tokenizer(model_name="t5-small"):
#   """
#   Loads a pre-trained T5 model and its corresponding tokenizer.

#   Args:
#     model_name: The name of the pre-trained T5 model to load.

#   Returns:
#     A tuple containing the loaded model and tokenizer.
#   """
#   tokenizer = T5Tokenizer.from_pretrained(model_name)
#   model = T5ForConditionalGeneration.from_pretrained(model_name)
#   return model, tokenizer

# def prepare_training_data(dataset, tokenizer):
#   """
#   Prepares the dataset for training by tokenizing the text.

#   Args:
#     dataset: The raw dataset containing text data.
#     tokenizer: The tokenizer to use for preprocessing.

#   Returns:
#     A tokenized dataset ready for training.
#   """
#   encodings = tokenizer(dataset['tokenized_answer'].tolist(), truncation=True, padding=True)
#   return Dataset.from_dict(encodings)
#   # Implement dataset creation logic (e.g., using Hugging Face Datasets)
#   pass

# def train_t5_model(model, tokenizer, dataset, output_dir, num_train_epochs=3, per_device_train_batch_size=2, warmup_steps=10, weight_decay=0.01, logging_dir=None):
#   """
#   Trains a T5 model on the given dataset.

#   Args:
#     model: The pre-trained T5 model.
#     tokenizer: The tokenizer used for preprocessing.
#     dataset: The training dataset.
#     output_dir: The directory to save the trained model and tokenizer.
#     num_train_epochs: The number of training epochs.
#     per_device_train_batch_size: The batch size for training.
#     warmup_steps: The number of warmup steps for the optimizer.
#     weight_decay: The weight decay for the optimizer.
#     logging_dir: The directory to save training logs.

#   Returns:
#     The trained model.
#   """

#   data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#   training_args = TrainingArguments(
#       output_dir=output_dir,
#       num_train_epochs=num_train_epochs,
#       per_device_train_batch_size=per_device_train_batch_size,
#       warmup_steps=warmup_steps,
#       weight_decay=weight_decay,
#       logging_dir=logging_dir,
#   )

#   trainer = Trainer(
#       model=model,
#       args=training_args,
#       data_collator=data_collator,
#       train_dataset=dataset
#   )

#   trainer.train()
#   return trainer.model

# # Example usage
# model, tokenizer = create_t5_model_and_tokenizer()
# train_data = prepare_training_data(df, tokenizer)

# output_dir = "/content/fine_tuned_T5_model"
# logging_dir = "/content/logging_dir"

# trained_model = train_t5_model(model, tokenizer, train_data, output_dir, logging_dir=logging_dir)


In [27]:
model.save_pretrained("/content/fine_tuned_T5_model")

In [28]:
import pandas as pd
test = pd.read_csv("/content/test_data.csv")
!pip install tqdm
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from tqdm import tqdm

In [31]:
qa_model_name = "t5-small"
!pip install transformers
from transformers import T5Tokenizer, AutoModelForQuestionAnswering # Import necessary classes
tokenizer = T5Tokenizer.from_pretrained(qa_model_name)
question_answering_model = AutoModelForQuestionAnswering.from_pretrained(qa_model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of T5ForQuestionAnswering were not initialized from the model checkpoint at t5-small and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
def predict_answer_using_question_answering_model(question_answering_model, tokenizer, question, device="cuda"):
  """
  Generates an answer to a given question using a pre-trained question answering model.

  Args:
      question_answering_model: The loaded question answering model.
      tokenizer: The tokenizer used to preprocess text for the model.
      question: The user-provided question string.
      device: The device to use for computation ("cuda" for GPU, "cpu" otherwise).

  Returns:
      The predicted answer string, or an empty string if no answer is found.
  """

  inputs = tokenizer.encode_plus(question, return_tensors="pt").to(device)
  with torch.no_grad():
    outputs = question_answering_model(**inputs)

    # Handle cases where no answer is found
    if outputs.start_logits.max().item() < 0.0:
      return ""

    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits) + 1
     # Return empty string if answer span is invalid
    if answer_start >= answer_end:
        return ""
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
  return answer

def evaluate_answer_predictions(ground_truth_answers, generated_answers):
  """
  Computes various evaluation metrics for a set of predicted answers to questions.

  Args:
      ground_truth_answers: A list of reference answer strings.
      generated_answers: A list of predicted answer strings.

  Returns:
      A tuple containing average BLEU, ROUGE-L, and F1 scores.
  """

  bleu_scores = []
  rouge_l_scores = []
  f1_scores = []

  rouge = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

  for ref, pred in zip(ground_truth_answers, generated_answers):
    # Handle cases where no answer is predicted
    if not pred:
      continue

    # BLEU score
    bleu_score = sentence_bleu([ref.split()], pred.split())
    bleu_scores.append(bleu_score)

    # ROUGE-L score
    rouge_l_score = rouge.score(ref, pred)['rougeL'].fmeasure
    rouge_l_scores.append(rouge_l_score)

    # F1-score
    ref_tokens = set(ref.split())
    pred_tokens = set(pred.split())
    common_tokens = ref_tokens.intersection(pred_tokens)

    if len(common_tokens) == 0:
      f1 = 0.0
    else:
      precision = len(common_tokens) / len(pred_tokens)
      recall = len(common_tokens) / len(ref_tokens)
      f1 = 2 * (precision * recall) / (precision + recall)
    f1_scores.append(f1)

  if not bleu_scores:
    avg_bleu = 0.0
  else:
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

  avg_rouge_l = sum(rouge_l_scores) / len(rouge_l_scores)
  avg_f1 = sum(f1_scores) / len(f1_scores)

  return avg_bleu, avg_rouge_l, avg_f1

# Example usage
device = "cuda" if torch.cuda.is_available() else "cpu"
question_answering_model.to(device)

questions = test.question
ground_truth_answers = test.answer

predicted_answers = [predict_answer_using_question_answering_model(question_answering_model, tokenizer, q, device) for q in questions]

avg_bleu, avg_rouge_l, avg_f1 = evaluate_answer_predictions(ground_truth_answers, predicted_answers)

print(f"Average BLEU Score: {avg_bleu}")
print(f"Average ROUGE-L Score: {avg_rouge_l}")
print(f"Average F1 Score: {avg_f1}")


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU Score: 0.00015367270092718344
Average ROUGE-L Score: 0.04644996788779249
Average F1 Score: 0.046729536955682056
